# Usando os truques de treinamento de redes neurais!

A documentação do Keras está em https://www.tensorflow.org/guide/keras

In [1]:
%matplotlib inline
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)

mpl.rc("axes", labelsize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

In [2]:
# GAMBIARRA: https://stackoverflow.com/questions/69687794/unable-to-manually-load-cifar10-dataset
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
import tensorflow as tf
from tensorflow import keras

tf.config.list_physical_devices()

2024-05-02 16:17:56.871800: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 16:17:58.455058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-02 16:18:01.978477: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-02 16:18:02.163073: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. 

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Lê o dataset CIFAR-10, https://keras.io/api/datasets/cifar10/
train_data, test_data = keras.datasets.cifar10.load_data()
X_train_full, y_train_full = train_data
X_test, y_test = test_data

Fonte: https://www.cs.toronto.edu/~kriz/cifar.html

<img src='cifar.png'></img>

O conjunto de treinamento tem $50000$ imagens de $32 \times 32$ pixels, com $3$ canais de cor. Portanto, o *array* `X_train_full` contendo esses dados tem formato $(\text{amostras}, \text{linhas}, \text{colunas}, \text{canais}) = (50000, 32, 32, 3)$

In [5]:
print(X_train_full.shape)

(50000, 32, 32, 3)


Cada canal de cor é representado como um inteiro entre $0$ e $255$, e isso cabe em um único *byte*. Para economizar espaço, o tipo de dados do *array* então será `uint8`, significando "inteiro sem-sinal de 8 bits". Ou para o povo do C/C++, o famoso `unsigned char`.

In [6]:
print(X_train_full.dtype)

uint8


A variável dependente `y_train_full` tem $50000$ amostras e apenas um valor por amostra. Para se manter consistente com os formatos de dados do *framework* Tensorflow, a variável dependente será representada por uma matriz-coluna de tamanho $(50000, 1)$ ao invés de um array simples de tamanho $(50000,)$

In [7]:
print(y_train_full.shape)

(50000, 1)


Como as classes são inteiros entre $0$ e $9$, um `uint8` basta para armazená-las

In [8]:
print(y_train_full.dtype)

uint8


Já o conjunto de testes tem $10000$ amostras, observe os tamanhos e tipos de dado de `X_test` e `y_test`:

In [9]:
print(X_test.shape, X_test.dtype)
print(y_test.shape, y_test.dtype)

(10000, 32, 32, 3) uint8
(10000, 1) uint8


O conjunto de teste, como de costume, é inviolável. Vamos dividir o conjunto de treinamento mais uma vez em duas partes: um conjunto que realmente será usado para treinamento, e outro para validação (que é o conjunto de testes de dentro do conjunto de treinamento pleno, e então não é inviolável).

Vamos separar $5000$ amostras para validação.

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full,
    y_train_full,
    test_size=5000,
)

print("X_train", X_train.shape, X_train.dtype)
print("y_train", y_train.shape, y_train.dtype)
print("X_valid", X_valid.shape, X_valid.dtype)
print("y_valid", y_valid.shape, y_valid.dtype)

X_train (45000, 32, 32, 3) uint8
y_train (45000, 1) uint8
X_valid (5000, 32, 32, 3) uint8
y_valid (5000, 1) uint8


Antes de mais nada, vamos criar *baselines* de desempenho para comparação: o classificador trivial e a Random Forest.

O classificador mais simples, mais trivial possível, é simplesmente dizer que toda amostra vem da mesma classe: a classe com maior representação no conjunto de treinamento. Vamos ver:

In [11]:
import pandas as pd

pd.Series(y_train.ravel()).value_counts(True).sort_values()

2    0.099400
3    0.099489
7    0.099800
5    0.099911
8    0.099933
9    0.100000
1    0.100289
4    0.100311
6    0.100333
0    0.100533
Name: proportion, dtype: float64

Aparentemente todas as classes estão representadas de modo aproximadamente igual, sendo a maior proporção igual a $10.05 \%$. Então vamos adotar como classe "oficial do nosso classificador trivial a classe $0$. Agora vamos ver o desempenho no conjunto de validação:

In [12]:
pd.Series(y_valid.ravel()).value_counts(True).sort_values()

0    0.0952
6    0.0970
4    0.0972
1    0.0974
9    0.1000
8    0.1006
5    0.1008
7    0.1018
3    0.1046
2    0.1054
Name: proportion, dtype: float64

Neste conjunto a classe $0$ tem, por acidente estatístico, a pior proporção! Paciência, a vida é assim! (Procure pelo fenômeno de "regressão para a média", muitas coisas da vida vão ficar mais claras...)

Portanto, o nosso desempenho do classificador trivial será uma acurácia de $9.5\%$.

Se você quiser uma estimativa melhor de desempenho do classificador trivial você pode fazer validação cruzada, ou mesmo estimar a estatística de acurácia usando *bootstrap* para obter uma distribuição do valor de acurácia. Mas nada disso será relevante: $9.5\%$ é muito baixo.

Vamos ver agora a RandomForest.

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [14]:
model = RandomForestClassifier(n_jobs=-1, random_state=42)
model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

RandomForestClassifier(n_jobs=-1, random_state=42)

In [15]:
y_pred = model.predict(X_valid.reshape(X_valid.shape[0], -1))
print(accuracy_score(y_valid.ravel(), y_pred))

0.454


Ok, esse é o *score* a ser vencido! (Será? Você pode pensar em alguma ressalva aqui?)

Vamos agora construir uma rede neural das antigas: nada profunda, e com função de ativação sigmoide. Vamos usar um `batch_size` grande, pois os itens do dataset são pequenos, não precisamos de batches muito pequenos (o tamanho padrão é 32 no TensorFlow). Veremos o desempenho:

In [16]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Input(shape=[32, 32, 3]))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation="sigmoid"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "inicial")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

2024-05-02 16:20:16.412603: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-02 16:20:16.413382: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-02 16:20:16.413991: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100


I0000 00:00:1714677620.779331   30369 service.cc:145] XLA service 0x76db50006da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714677620.779383   30369 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1050 Ti with Max-Q Design, Compute Capability 6.1
2024-05-02 16:20:20.800275: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-02 16:20:20.852317: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


38/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1273 - loss: 2.3797

I0000 00:00:1714677621.539362   30369 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.1306 - loss: 2.3671 - val_accuracy: 0.1816 - val_loss: 2.2291
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1899 - loss: 2.2075 - val_accuracy: 0.1926 - val_loss: 2.1760
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2117 - loss: 2.1576 - val_accuracy: 0.2230 - val_loss: 2.1351
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2325 - loss: 2.1230 - val_accuracy: 0.2452 - val_loss: 2.1163
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2524 - loss: 2.0881 - val_accuracy: 0.2414 - val_loss: 2.0857
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2650 - loss: 2.0627 - val_accuracy: 0.2586 - val_loss: 2.0597
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2752 - loss: 2.0468 - val_accuracy: 0.2730 - val_loss: 2.0493
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2861 - loss: 2.0178 - val_accuracy: 0.2764 - val_loss: 2.

[1.795331597328186, 0.3546000123023987]

Vamos partir para uma rede profunda agora. Vamos criar $10$ camadas *fully-connected*, função de ativação "elu", inicialização de pesos adequada para a respectiva função de ativação, e otimizador "adam". Vamos ver se melhora:

In [17]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Input(shape=[32, 32, 3]))
model.add(keras.layers.Flatten())
for _ in range(10):
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "profunda")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
27/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0962 - loss: 230.3988

I0000 00:00:1714677672.341066   30369 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 104 bytes spill stores, 132 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads



44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0981 - loss: 195.6609

I0000 00:00:1714677673.606981   30369 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 104 bytes spill stores, 132 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads



44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.0982 - loss: 194.0293 - val_accuracy: 0.1078 - val_loss: 42.0386
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1417 - loss: 22.9680 - val_accuracy: 0.1738 - val_loss: 4.7340
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1852 - loss: 3.9990 - val_accuracy: 0.1972 - val_loss: 2.8522
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1851 - loss: 3.2475 - val_accuracy: 0.1608 - val_loss: 3.4328
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1725 - loss: 4.0248 - val_accuracy: 0.1844 - val_loss: 5.8375
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1811 - loss: 3.8296 - val_accuracy: 0.2052 - val_loss: 2.4131
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2237 - loss: 2.3496 - val_accuracy: 0.2158 - val_loss: 2.2664
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2341 - loss: 2.2552 - val_accuracy: 0.2206 - val_loss

[1.7130221128463745, 0.40119999647140503]

Muito melhor! Agora acertamos em torno de $43\%$ das classes!

A técnica de "batch normalization" pode melhorar ainda mais a convergência do modelo, vamos experimentar:

In [18]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Input(shape=[32, 32, 3]))
model.add(keras.layers.Flatten())
for _ in range(10):
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "batchnorm")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
 7/44 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1786 - loss: 2.6290 

I0000 00:00:1714677728.686144   30366 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 104 bytes spill stores, 132 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads



44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2900 - loss: 2.1049

I0000 00:00:1714677731.337117   30366 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 104 bytes spill stores, 132 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads



44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 94ms/step - accuracy: 0.2913 - loss: 2.0994 - val_accuracy: 0.1636 - val_loss: 4.7680
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4385 - loss: 1.5740 - val_accuracy: 0.3012 - val_loss: 2.4144
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4815 - loss: 1.4458 - val_accuracy: 0.3906 - val_loss: 1.8288
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5155 - loss: 1.3554 - val_accuracy: 0.4270 - val_loss: 1.6623
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5441 - loss: 1.2801 - val_accuracy: 0.4444 - val_loss: 1.5956
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5682 - loss: 1.2142 - val_accuracy: 0.4502 - val_loss: 1.5886
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5887 - loss: 1.1624 - val_accuracy: 0.4538 - val_loss: 1.6120
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6024 - loss: 1.1245 - val_accuracy: 0.4504 - val_los

[4.772695064544678, 0.44620001316070557]

Uau. Isso sim que é *overfitting*! Para resolver o problema do *overfitting* vamos adotar o EarlyStopping:

In [19]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "earlystopping")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=[32, 32, 3]))
model.add(keras.layers.Flatten())
for _ in range(10):
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - accuracy: 0.2841 - loss: 2.1138 - val_accuracy: 0.1628 - val_loss: 4.4832
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4346 - loss: 1.5801 - val_accuracy: 0.3116 - val_loss: 2.2054
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4823 - loss: 1.4488 - val_accuracy: 0.3962 - val_loss: 1.7884
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5168 - loss: 1.3549 - val_accuracy: 0.4382 - val_loss: 1.6251
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5464 - loss: 1.2811 - val_accuracy: 0.4562 - val_loss: 1.5735
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5692 - loss: 1.2170 - val_accuracy: 0.4642 - val_loss: 1.5568
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5876 - loss: 1.1646 - val_accuracy: 0.4602 - val_loss: 1.5673
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6036 - loss: 1.1225 - val_accuracy: 0.

[1.753298282623291, 0.4551999866962433]

Melhorou mais! Mas ainda temos o problema do *overfitting*, vamos tentar a técnica do "drop-out" para ver se melhora:

In [20]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "dropout")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=[32, 32, 3]))
model.add(keras.layers.Flatten())
for _ in range(10):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
 6/44 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1287 - loss: 2.9146  

I0000 00:00:1714677856.551270   30367 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 104 bytes spill stores, 132 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads



44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - accuracy: 0.1911 - loss: 2.4607

I0000 00:00:1714677873.232256   30368 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 104 bytes spill stores, 132 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads



44/44 ━━━━━━━━━━━━━━━━━━━━ 38s 411ms/step - accuracy: 0.1921 - loss: 2.4549 - val_accuracy: 0.1552 - val_loss: 5.3983
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3261 - loss: 1.8641 - val_accuracy: 0.2804 - val_loss: 3.0079
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3665 - loss: 1.7570 - val_accuracy: 0.3546 - val_loss: 2.1136
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3944 - loss: 1.6859 - val_accuracy: 0.4126 - val_loss: 1.7884
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4085 - loss: 1.6403 - val_accuracy: 0.4382 - val_loss: 1.6351
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4288 - loss: 1.5947 - val_accuracy: 0.4522 - val_loss: 1.5821
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4394 - loss: 1.5701 - val_accuracy: 0.4602 - val_loss: 1.5286
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4533 - loss: 1.5334 - val_accuracy: 0.4642 - val

[1.3462516069412231, 0.5335999727249146]

Melhorou ainda mais! E agora não tem mais tanto overfitting! Já que é assim, vamos aumentar ainda mais a rede e ver se melhora: vamos colocar $20$ camadas!

In [21]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "mais_profunda")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=[32, 32, 3]))
model.add(keras.layers.Flatten())
for _ in range(20):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 99s 916ms/step - accuracy: 0.1241 - loss: 2.6786 - val_accuracy: 0.1040 - val_loss: 8.4038
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.2187 - loss: 2.1123 - val_accuracy: 0.1760 - val_loss: 4.8543
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.2741 - loss: 1.9608 - val_accuracy: 0.2540 - val_loss: 3.2074
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3081 - loss: 1.8822 - val_accuracy: 0.3144 - val_loss: 2.4001
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3372 - loss: 1.8256 - val_accuracy: 0.3496 - val_loss: 2.0789
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3536 - loss: 1.7733 - val_accuracy: 0.3782 - val_loss: 1.8927
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3733 - loss: 1.7364 - val_accuracy: 0.4054 - val_loss: 1.7641
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3861 - loss: 1.6940 - val_accuracy: 

[1.3632358312606812, 0.5281999707221985]

Parece que não melhorou muito, estava bom do jeito anterior. Para terminar, vamos usar um escalonador de taxa de aprendizado para melhorar a velocidade de convergência do nosso modelo.

In [22]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "escalonador_mais_dropout")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=[32, 32, 3]))
model.add(keras.layers.Flatten())
for _ in range(10):
    model.add(keras.layers.Dropout(rate=0.3))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

s = 90 * len(X_train) // 1024  # number of steps in 90 epochs (batch size 1024)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
 6/44 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1084 - loss: 2.8506  

I0000 00:00:1714678105.709876   30367 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 104 bytes spill stores, 132 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads



44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step - accuracy: 0.1575 - loss: 2.4082

I0000 00:00:1714678121.726827   30369 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 104 bytes spill stores, 132 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 24 bytes spill stores, 24 bytes spill loads



44/44 ━━━━━━━━━━━━━━━━━━━━ 38s 409ms/step - accuracy: 0.1583 - loss: 2.4035 - val_accuracy: 0.1064 - val_loss: 66.2147
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.2727 - loss: 1.9616 - val_accuracy: 0.2272 - val_loss: 6.9713
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3064 - loss: 1.8929 - val_accuracy: 0.3020 - val_loss: 2.7238
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3252 - loss: 1.8407 - val_accuracy: 0.3520 - val_loss: 2.1051
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3481 - loss: 1.7948 - val_accuracy: 0.3678 - val_loss: 1.8456
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3661 - loss: 1.7626 - val_accuracy: 0.3870 - val_loss: 1.7279
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3734 - loss: 1.7382 - val_accuracy: 0.3934 - val_loss: 1.6835
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.3799 - loss: 1.7216 - val_accuracy: 0.4112 - va

[1.396950602531433, 0.492000013589859]

In [23]:
model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5166 - loss: 1.3499


[1.3582535982131958, 0.5127000212669373]

**Atividade**: Refaça a atividade da aula anterior (Fashion MNIST) mas tente usar os truques de melhoramento de desempenho das redes neurais vistos aqui nesta aula.